<a href="https://colab.research.google.com/github/aldebaro/ai6g/blob/main/solutions_03_channel_estimation_hybrid_mimo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Inteligência Artificial e Aprendizado de Máquina Aplicados a Redes 5G e 6G**.
*Aldebaro Klautau* (UFPA). Minicurso 5 do SBrT - 25 de setembro de 2022.

To install and import all the Needed packages

In [ ]:
# Clone the repository if running in Colab and install all the dependencies
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive/')
    import sys
    import os
    try:
      !git clone https://github.com/aldebaro/ai6g.git
    except:
      print("ai6g is already in the contents")
    %cd ai6g
    !ln -s /content/drive/MyDrive/ai6g_files/files_03_channel/* ./files_03_channel
    !pip install -r requirements.txt

In [1]:
#!pip install tensorflow==2.10.0 keras-tuner==1.1.3 numpy h5py==3.7.0
import h5py 
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (
    Dense,
    Reshape,
    Conv2D,
    Dropout,
    Flatten,
    BatchNormalization,
)
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
)


In [2]:

FILES={
    1:"./files_03_channel/Pilots_SNR_(-5dB)_64x16(80pilots)_single_sub.hdf5",
    2:"./files_03_channel/Pilots_SNR_(-5dB)_8x4(80pilots)_single_sub.hdf5",
    3:"./files_03_channel/Pilots_SNR_(0dB)_8x4(80pilots)_single_sub.hdf5",
    "64x16":"./files_03_channel/Channels_64x16_single_sub.hdf5",
    "8x4":"./files_03_channel/Channels_8x4_single_sub.hdf5",
}


Change the following values according to the dataset

In [3]:
CHOSEN_DATASET = 2
BATCH_SIZE = 32
EPOCHS = 1000


# Change these values acording to the dataset
NR = 8 
NT = 4 
LR = 4 
CHANNELS_USED_TRAIN = 9000
SUBCARIERS_USED = 1
TOTAL_CHANNELS = 10000
NUM_SYM_PILOTS = 80
CHANNELS_USED_TEST = TOTAL_CHANNELS - CHANNELS_USED_TRAIN



input_shape = (NUM_SYM_PILOTS * LR, SUBCARIERS_USED)
output_shape = (NR, NT, SUBCARIERS_USED)

SELECTING THE DATASET

In [4]:
if CHOSEN_DATASET == 1:
    CHOSEN_CHANNELS = FILES["64x16"]
elif CHOSEN_DATASET > 1 and CHOSEN_DATASET <=3:
    CHOSEN_CHANNELS = FILES["8x4"]


# Directory of the Channels Matrix file
CHANNELS_FILE = h5py.File(CHOSEN_CHANNELS, "r")


# Directory of the Pilots Matrix file
PILOTS_FILE = h5py.File(FILES[CHOSEN_DATASET], "r")

This function will be used as a metric in keras to calculate the NMSE(Normalized Mean Squared Error) for each train or validation example that will be iterated in the training process, providing clear metrics to the whole process and enabling the use of early_stopping.

In [5]:
# Declaring the function to calculate NMSE for each batch
def NMSEtrainComplex(y_true, y_pred):
    sub = y_pred[:, :, :] - y_true[:, :, :]
    H_k = y_true[:, :, :]
    nmse = tf.norm(sub, ord="fro", axis=(1, 2)) ** 2
    den = tf.norm(H_k, ord="fro", axis=(1, 2)) ** 2

    result = (nmse / den)

    return 10*tf.experimental.numpy.log10(result)

Now the NN model will be crated, a chain of Dense layers is being used with relu activation.

In [6]:
model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(BatchNormalization())
model.add(Dense((512), activation="relu"))
model.add(Dropout(0.1))
model.add(Dense((512), activation="relu"))
model.add(Dropout(0.1))
model.add(Dense((512), activation="relu"))
model.add(Dropout(0.1))
model.add(Dense((512), activation="relu"))
model.add(Dropout(0.1))
model.add(Dense((512), activation="relu"))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(np.prod(output_shape), activation="linear"))
model.add(Reshape(output_shape))
model.compile(
    loss="mean_squared_error", optimizer="adam", metrics=[NMSEtrainComplex]
    )

Train the Model

In [ ]:

training_inputData = PILOTS_FILE["pilots"][:CHANNELS_USED_TRAIN, :, :SUBCARIERS_USED]
training_outputData = CHANNELS_FILE["channels"][:CHANNELS_USED_TRAIN, :, :, :SUBCARIERS_USED]


history = model.fit(
        training_inputData,
        training_outputData,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        verbose=1,
        shuffle=True,
        validation_split=0.2,
        callbacks=[
            EarlyStopping(
                monitor="val_NMSEtrainComplex",
                min_delta=5e-3,
                patience=40,
                restore_best_weights=True,
            ),
            ReduceLROnPlateau(
                factor=0.5,
                min_delta=5e-2,
                patience=20,
                cooldown=5,
                verbose=1,
                min_lr=1e-7,
            ),
        ],
    )
model.save("./files_03_channel/model")

Epoch 1/1000
225/225 [==============================] - 12s 17ms/step - loss: 0.6909 - NMSEtrainComplex: -0.0017 - val_loss: 0.3561 - val_NMSEtrainComplex: -2.7802 - lr: 0.0010
Epoch 2/1000
225/225 [==============================] - 3s 13ms/step - loss: 0.3860 - NMSEtrainComplex: -2.3424 - val_loss: 0.2729 - val_NMSEtrainComplex: -4.1251 - lr: 0.0010
Epoch 3/1000
225/225 [==============================] - 3s 13ms/step - loss: 0.3010 - NMSEtrainComplex: -3.4995 - val_loss: 0.2304 - val_NMSEtrainComplex: -5.0783 - lr: 0.0010
Epoch 4/1000
225/225 [==============================] - 4s 16ms/step - loss: 0.2526 - NMSEtrainComplex: -4.3537 - val_loss: 0.2007 - val_NMSEtrainComplex: -5.7514 - lr: 0.0010
Epoch 5/1000
225/225 [==============================] - 3s 15ms/step - loss: 0.2207 - NMSEtrainComplex: -4.9892 - val_loss: 0.1848 - val_NMSEtrainComplex: -6.1100 - lr: 0.0010
Epoch 6/1000
225/225 [==============================] - 4s 16ms/step - loss: 0.2000 - NMSEtrainComplex: -5.4259 - val_l

225/225 [==============================] - 4s 16ms/step - loss: 0.0841 - NMSEtrainComplex: -8.9700 - val_loss: 0.1143 - val_NMSEtrainComplex: -8.3340 - lr: 5.0000e-04
Epoch 47/1000
225/225 [==============================] - 3s 15ms/step - loss: 0.0849 - NMSEtrainComplex: -8.9255 - val_loss: 0.1139 - val_NMSEtrainComplex: -8.3882 - lr: 5.0000e-04
Epoch 48/1000
225/225 [==============================] - 3s 15ms/step - loss: 0.0845 - NMSEtrainComplex: -8.9569 - val_loss: 0.1152 - val_NMSEtrainComplex: -8.3271 - lr: 5.0000e-04
Epoch 49/1000
225/225 [==============================] - 3s 16ms/step - loss: 0.0843 - NMSEtrainComplex: -8.9562 - val_loss: 0.1147 - val_NMSEtrainComplex: -8.3270 - lr: 5.0000e-04
Epoch 50/1000
225/225 [==============================] - 3s 15ms/step - loss: 0.0832 - NMSEtrainComplex: -9.0141 - val_loss: 0.1144 - val_NMSEtrainComplex: -8.3750 - lr: 5.0000e-04
Epoch 51/1000
225/225 [==============================] - 3s 15ms/step - loss: 0.0822 - NMSEtrainComplex: -9.0

213/225 [===========================>..] - ETA: 0s - loss: 0.0701 - NMSEtrainComplex: -9.7047

TEST MODEL

In [ ]:
mlModel = load_model("./files_03_channel/model", custom_objects={"NMSEtrainComplex": NMSEtrainComplex})

TEST_INPUT = PILOTS_FILE["pilots"][CHANNELS_USED_TRAIN:, :, :SUBCARIERS_USED]
TEST_OUTPUT = CHANNELS_FILE["channels"][CHANNELS_USED_TRAIN:, :, :, :SUBCARIERS_USED]

results = mlModel.evaluate(TEST_INPUT,TEST_OUTPUT)





## Authors

- Prof. Francisco Muller - LASSE/UFPA
- Daniel Oliveira - LASSE/UFPA
- Claudio Mello - LASSE/UFPA